In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
### Merge Employment data to National Student Survey data

In [2]:
aorig = pd.read_csv('he_choose_detail.csv', header=0, index_col=0)

print(aorig.shape)
aorig.head()

(66177, 9)


,UKPRN,Provider,Subject Code,Subject,Level,Question Number,Actual value,Response,Country
0,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),First degree,Scale01,0.9394,99,Scotland
1,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),First degree,Scale02,0.9158,99,Scotland
2,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),First degree,Scale03,0.6944,99,Scotland
3,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),First degree,Scale04,0.8838,99,Scotland
4,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),First degree,Scale05,0.8636,99,Scotland


## Prep survey data

In [3]:
aedit = aorig.copy()

aedit = aedit[aedit.Level == "First degree"]
aedit.drop(['Level', 'Response'], axis=1, inplace=True)
aedit.rename(columns={'Question Number':'Cat', 'Actual value':'Value'}, inplace=True)

print(aedit.shape)
aedit.head(2)

(55781, 7)


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country
0,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Scale01,0.9394,Scotland
1,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Scale02,0.9158,Scotland


In [4]:
aedit.Cat.unique()

array(['Scale01', 'Scale02', 'Scale03', 'Scale04', 'Scale05', 'Scale06',
       'Scale07', 'Scale08', 'Q26', 'Q27'], dtype=object)

In [5]:
# Change Overall Sat and Unions for reorder
aedit.replace('Q27', '03Q27', inplace=True)
aedit.replace('Q26', 'ZQ26', inplace=True)

print(aedit.Cat.unique())

['Scale01' 'Scale02' 'Scale03' 'Scale04' 'Scale05' 'Scale06' 'Scale07'
 'Scale08' 'ZQ26' '03Q27']


## Prep Salary data

In [6]:
sorig = pd.read_csv('Unistats/SALARY.csv', header=0)

print(sorig.shape)
sorig.head(2)

(46627, 18)


,PUBUKPRN,UKPRN,KISCOURSEID,KISMODE,SALUNAVAILREASON,SALPOP,SALRESP_RATE,SALAGG,SALSBJ,LDLQ,LDMED,LDUQ,LQ,MED,UQ,INSTLQ,INSTMED,INSTUQ
0,10000047,10001143,PSSFDOPTDIS,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000055,10000055,AB20,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Very few uni-subject combos had 2 values... 
# We drop the 2nd as likely associated other undergrad
sedit = sorig.copy()

sedit = sedit[sedit.SALSBJ.isnull() == False]
sedit = sedit[sedit.KISMODE == 1] # keep full-time
sedit = sedit[['UKPRN', 'SALSBJ', 'INSTMED']]
sedit.drop_duplicates(subset=['UKPRN', 'SALSBJ'], keep='first', inplace=True)

print(sedit.shape)
sedit.head(5)

(3960, 3)


,UKPRN,SALSBJ,INSTMED
23,10000248,CAH21-01-06,26000.0
27,10000291,CAH21-01,17000.0
29,10000291,CAH21-01-03,17000.0
32,10000291,CAH11-01-06,20000.0
33,10000291,CAH21-01-08,19000.0


In [8]:
sedit['SBJLEN'] = sedit.SALSBJ.apply(len)

sedit.head(2)

,UKPRN,SALSBJ,INSTMED,SBJLEN
23,10000248,CAH21-01-06,26000.0,11
27,10000291,CAH21-01,17000.0,8


In [9]:
sedit = sedit[sedit.SBJLEN == 11]
sedit.drop(['SBJLEN'], axis=1, inplace=True)
sedit['Cat'] = '01SalMedian'
sedit.rename(columns={'SALSBJ':'Subject Code','INSTMED':'Value'}, inplace=True)

sedit.head(2)

,UKPRN,Subject Code,Value,Cat
23,10000248,CAH21-01-06,26000.0,01SalMedian
29,10000291,CAH21-01-03,17000.0,01SalMedian


## Add Salary Uni name, Subject and Country

In [10]:
print(f'Schools in orig not in salary: {len(set(aedit.UKPRN) - set(sedit.UKPRN))}')
print(f'Schools in salary not in orig: {len(set(sedit.UKPRN) - set(aedit.UKPRN))}')

Schools in orig not in salary: 102
Schools in salary not in orig: 20


In [11]:
# remove uni from salary not in orig
sedit = sedit[sedit.UKPRN.isin((set(sedit.UKPRN) - set(aedit.UKPRN))) == False]
print(f'Schools in salary not in orig: {len(set(sedit.UKPRN) - set(aedit.UKPRN))}')

Schools in salary not in orig: 0


In [12]:
print(f"Subjects in orig not in salary: {len(set(aedit['Subject Code']) - set(sedit['Subject Code']))}")
print(f"Subjects in salary not in orig: {len(set(sedit['Subject Code']) - set(aedit['Subject Code']))}")

Subjects in orig not in salary: 1
Subjects in salary not in orig: 19


In [13]:
# remove subjects from salary not in orig
sedit = sedit[sedit['Subject Code'].isin((set(sedit['Subject Code']) - set(aedit['Subject Code']))) == False]
print(f"Subjects in salary not in orig: {len(set(sedit['Subject Code']) - set(aedit['Subject Code']))}")

Subjects in salary not in orig: 0


In [14]:
sedit.head(2)

,UKPRN,Subject Code,Value,Cat
23,10000248,CAH21-01-06,26000.0,01SalMedian
29,10000291,CAH21-01-03,17000.0,01SalMedian


In [15]:
anamecn = aedit.copy()
asbj = aedit.copy()

anamecn = anamecn[['UKPRN', 'Provider', 'Country']]
asbj = asbj[['Subject Code', 'Subject']]

anamecn.drop_duplicates(keep='first', inplace=True)
asbj.drop_duplicates(keep='first', inplace=True)

sedit = pd.merge(sedit, anamecn, on='UKPRN')
sedit = pd.merge(sedit, asbj, on='Subject Code')
sedit = sedit[list(aedit.columns)]

print(sedit.info())
sedit.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2625 entries, 0 to 2624
Data columns (total 7 columns):
UKPRN           2625 non-null int64
Provider        2625 non-null object
Subject Code    2625 non-null object
Subject         2625 non-null object
Cat             2625 non-null object
Value           2625 non-null float64
Country         2625 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 164.1+ KB
None


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country
0,10000248,ALRA,CAH21-01-06,Drama,01SalMedian,26000.0,England
1,10000291,Anglia Ruskin University Higher Corporation,CAH21-01-06,Drama,01SalMedian,14000.0,England


## Prep Employment data

In [16]:
eorig = pd.read_csv('Unistats/EMPLOYMENT.csv', header=0)

print(eorig.shape)
eorig.head(2)

(47994, 15)


,PUBUKPRN,UKPRN,KISCOURSEID,KISMODE,EMPUNAVAILREASON,EMPPOP,EMPRESP_RATE,EMPAGG,EMPSBJ,WORKSTUDY,STUDY,ASSUNEMP,BOTH,NOAVAIL,WORK
0,10000047,10001143,PSSFDOPTDIS,1,0,20.0,70.0,14.0,NaN,100.0,40.0,0.0,35.0,0.0,30.0
1,10000055,10000055,AB20,1,0,15.0,100.0,14.0,NaN,95.0,80.0,5.0,5.0,0.0,5.0


In [17]:
eedit = eorig.copy()

eedit = eedit[eedit.EMPSBJ.isnull() == False]
eedit = eedit[eedit.KISMODE == 1] # keep full-time
eedit = eedit[['UKPRN', 'EMPSBJ', 'WORK']]
eedit.drop_duplicates(subset=['UKPRN', 'EMPSBJ'], keep='first', inplace=True)

print(eedit.shape)
eedit.head(5)

(4892, 3)


,UKPRN,EMPSBJ,WORK
2,10000055,CAH06-01-01,75.0
4,10000055,CAH10-01,30.0
5,10000055,CAH17-01,20.0
7,10000055,CAH06-01,15.0
12,10000163,CAH02-03,65.0


In [18]:
eedit['SBJLEN'] = eedit.EMPSBJ.apply(len)

eedit.head(2)

,UKPRN,EMPSBJ,WORK,SBJLEN
2,10000055,CAH06-01-01,75.0,11
4,10000055,CAH10-01,30.0,8


In [19]:
eedit = eedit[eedit.SBJLEN == 11]
eedit.drop(['SBJLEN'], axis=1, inplace=True)
eedit['Cat'] = '02Employment'
eedit['WORK'] = eedit['WORK']/100
eedit.rename(columns={'EMPSBJ':'Subject Code','WORK':'Value'}, inplace=True)

eedit.head(2)

,UKPRN,Subject Code,Value,Cat
2,10000055,CAH06-01-01,0.75,02Employment
23,10000248,CAH21-01-06,0.95,02Employment


## Add Employment Uni name, Subject and Country

In [20]:
print(f'Schools in orig not in employment: {len(set(aedit.UKPRN) - set(eedit.UKPRN))}')
print(f'Schools in employment not in orig: {len(set(eedit.UKPRN) - set(aedit.UKPRN))}')

Schools in orig not in employment: 70
Schools in employment not in orig: 76


In [21]:
# visualize uni in orig not in employment
aedit[aedit.UKPRN.isin((set(aedit.UKPRN) - set(eedit.UKPRN)))].Provider.unique()[:10]

array(['ACM Guildford Limited', 'AECC University College',
       'All Nations Christian College Limited',
       'Architectural Association (Incorporated)',
       'Arden University Limited', 'Arts Educational Schools(The)',
       'Assemblies of God Incorporated',
       'Backstage Academy (training) Ltd', 'BCNO Limited', 'BIMM Limited'],
      dtype=object)

In [22]:
# remove uni from employment not in orig
eedit = eedit[eedit.UKPRN.isin((set(eedit.UKPRN) - set(aedit.UKPRN))) == False]
print(f'Schools in employment not in orig: {len(set(eedit.UKPRN) - set(aedit.UKPRN))}')

Schools in employment not in orig: 0


In [23]:
print(f"Subjects in orig not in Employment: {len(set(aedit['Subject Code']) - set(eedit['Subject Code']))}")
print(f"Subjects in Employment not in orig: {len(set(eedit['Subject Code']) - set(aedit['Subject Code']))}")

Subjects in orig not in Employment: 0
Subjects in Employment not in orig: 29


In [24]:
# remove subjects from Employment not in orig
eedit = eedit[eedit['Subject Code'].isin((set(eedit['Subject Code']) - set(aedit['Subject Code']))) == False]
print(f"Subjects in Employment not in orig: {len(set(eedit['Subject Code']) - set(aedit['Subject Code']))}")

Subjects in Employment not in orig: 0


In [25]:
eedit = pd.merge(eedit, anamecn, on='UKPRN')
eedit = pd.merge(eedit, asbj, on='Subject Code')
eedit = eedit[list(aedit.columns)]

print(eedit.info())
eedit.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3454 entries, 0 to 3453
Data columns (total 7 columns):
UKPRN           3454 non-null int64
Provider        3454 non-null object
Subject Code    3454 non-null object
Subject         3454 non-null object
Cat             3454 non-null object
Value           3454 non-null float64
Country         3454 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 215.9+ KB
None


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country
0,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,02Employment,0.75,England
1,10000415,Askham Bryan College,CAH06-01-01,Animal science,02Employment,0.70,England


## Append Salary and Employment data

In [26]:
print(f'Orig data shape: {aedit.shape}')
print(f'Salary data shape: {sedit.shape}')
print(f'Employment data shape: {eedit.shape}')
print(f'Anticipated new length: {aedit.shape[0] + sedit.shape[0] + eedit.shape[0]}')

Orig data shape: (55781, 7)
Salary data shape: (2625, 7)
Employment data shape: (3454, 7)
Anticipated new length: 61860


In [32]:
new_merge = aedit.append(sedit)
new_merge = new_merge.append(eedit)
new_merge.sort_values(by=['UKPRN', 'Subject Code', 'Cat', 'Country'], inplace=True)
new_merge.reset_index(drop=True, inplace=True)

print(new_merge.shape)
new_merge.head()

(61860, 7)


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country
0,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,02Employment,0.7500,England
1,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,03Q27,0.9167,England
2,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale01,0.8333,England
3,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale02,0.9444,England
4,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale03,0.9583,England


In [35]:
new_merge.Cat.value_counts()

Scale03         5590
Scale01         5590
Scale07         5590
Scale05         5590
Scale04         5590
Scale02         5590
Scale06         5590
Scale08         5587
03Q27           5580
ZQ26            5484
02Employment    3454
01SalMedian     2625
Name: Cat, dtype: int64

In [36]:
new_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61860 entries, 0 to 61859
Data columns (total 7 columns):
UKPRN           61860 non-null int64
Provider        61860 non-null object
Subject Code    61860 non-null object
Subject         61860 non-null object
Cat             61860 non-null object
Value           61860 non-null float64
Country         61860 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 3.3+ MB


In [37]:
# new_merge.to_csv('he_choose_salemp.csv')

## Extra code

In [ ]:
# # code that identified repeats in uni-subject combo
# eedit['Counter'] = 1

# e_piv = pd.pivot_table(eedit,  values='Counter', index=['UKPRN', 'Subject'], aggfunc=np.sum)

# e_flat = pd.DataFrame(e_piv.to_records())

# print(len(e_flat[e_flat.Counter == 2].Subject.unique()))
# e_flat[e_flat.Counter == 2].Subject.unique()

In [ ]:
test = new_merge.copy()
test2 = new_merge.copy()

In [ ]:
test = test[(test.Provider == 'The Trafford College Group') & (test.Subject == 'Education')]

In [ ]:
test

In [ ]:
one='01SalMedian'
two='Scale01'
three='Scale05'

In [ ]:
test2 = test2[(test2.Country == 'England') & (test2.Subject == 'Law')]
test2 = test2[(test2.Cat == one) | (test2.Cat == two) | (test2.Cat == three)]

print(test2.shape)
test2.head(5)

In [ ]:
test2_vc = test2.Provider.value_counts()

print(len(test2_vc[test2_vc == 3]))
print(len(test2_vc[test2_vc == 2]))
print(len(test2_vc[test2_vc == 1]))

In [ ]:
test2_list = test2_vc[test2_vc == 3].index.tolist()

test2_list[:10]

In [ ]:
test2f = test2[test2.Provider.isin(test2_list)]

test2f.shape

In [ ]:
192/3

In [ ]:
test2f[test2f.Provider == 'Anglia Ruskin University Higher Corporation']

In [ ]:
test = new_merge.copy()
test = test[(test.Country == 'England') & (test.Subject == 'Sociology')]
# test = test[test.Provider == 'The University of Bath']

test[test.Cat == one].Provider.values

In [ ]:
# # identified that repeats likely due to other undergrad
# print(len(aorig[aorig.Level != 'First degree'].Subject.unique()))
# aorig[aorig.Level != 'First degree'].Subject.unique()